In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing

def run(fold):
    df = pd.read_csv("/Users/pranesh/coding2/MachineLearningFrom0/datasets/adult.csv")
    num_cols = ["fnlwgt", "age", "education-num", "capital-gain", "capital-loss", "hours-per-week"]
    
    df = df.drop(num_cols, axis = 1)
    
    target_mapping={
        "<=50k":0,
        ">50k":1
    }
    df.loc[:, "income"] = df.income.map(target_mapping)
    features =[ f for f in df.columns if f not in ("kfold", "income")]
    
    for col in features:
        df.loc[:, col] = df[col].astype(str).fillna("NONE")
        
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    ohe = preprocessing.OneHotEncoder()
    full_data = pd.concat([df_train[features], df_valid[features]], axis=0)
    ohe.fit(full_data[features])
    x_train = ohe.transform(df_train[features])
    x_valid = ohe.transform(df_valid[features])
    
    model = linear_model.LogisticRegression()
    valid_preds = model.predict_proba(x_valid)[:, 1]
    auc = metrics.roc_auc_score(df_valid.income, valid_preds)
    print(f"Fold = {fold}, AUC = {auc}")
    
    if __name__ == "__main__":
        for fold_ in range(5):
            run(fold_)